In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from keras.layers.advanced_activations import LeakyReLU

In [2]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalization
X_train = X_train.astype(np.float32)/127.5 -1

# Reshape
X_train = X_train.reshape(60000,784)

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
from keras.optimizers import Adam

adam = Adam(lr = 0.0002, beta_1 = 0.5)

In [5]:
noise_dim = 10

generator = Sequential(name='generator')
generator.add(Dense(256, input_shape=(noise_dim,), activation='relu'))
generator.add(Dense(512, activation='relu'))
generator.add(Dense(1024, activation='relu'))
generator.add(Dense(784, activation='tanh'))

generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               2816      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_3 (Dense)              (None, 784)               803600    
Total params: 1,463,312
Trainable params: 1,463,312
Non-trainable params: 0
_________________________________________________________________


In [6]:
from keras.initializers import RandomNormal

discriminator = Sequential(name='discriminator')
discriminator.add(Dense(1024, input_shape=(784,), kernel_initializer=RandomNormal(stddev=0.02), activation='relu'))
discriminator.add(Dropout(0.5))
discriminator.add(Dense(512, activation='relu'))
discriminator.add(Dropout(0.5))
discriminator.add(Dense(256, activation='relu'))
discriminator.add(Dropout(0.5))
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1024)              803840    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)               

In [7]:
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

In [8]:
discriminator.trainable = False

gan_input = Input(shape =(noise_dim,))
x = generator(gan_input)
output = discriminator(x)

gan = Model(gan_input, output, name='gan')